In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [65]:
from fastai.vision import *
from fastai.metrics import accuracy


from torch.autograd import Variable
from torchvision import models
import numpy as np
import matplotlib.pyplot as plt
import skimage.transform

In [3]:
PATH = "/scratch/as3ek/chrc_data_case_preserved_1000_normal/"
sz=512
tfms = get_transforms(do_flip=True, flip_vert=True, max_zoom=1.1)

In [4]:
data = (ImageItemList.from_folder(PATH)
        .split_by_folder()          
        .label_from_folder()
        .transform(tfms, size=sz)
        .databunch(bs=16)
        .normalize(imagenet_stats))

In [5]:
class AdaptiveConcatPool2d(nn.Module):
    def __init__(self, sz=None):
        super().__init__()
        sz = sz or (1,1)
        self.ap = nn.AdaptiveAvgPool2d(sz)
        self.mp = nn.AdaptiveMaxPool2d(sz)
    
    def forward(self, x): 
        return torch.cat([self.mp(x), self.ap(x)], 1)
    
    
custom_head = nn.Sequential(
                            AdaptiveConcatPool2d(),
                            Flatten(), 
                            nn.BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                            nn.Dropout(p=0.25),
                            nn.Linear(in_features=4096, out_features=512, bias=True),
                            nn.ReLU(),
                            nn.BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                            nn.Dropout(p=0.5),
                            nn.Linear(in_features=512, out_features=10, bias=True),
                            nn.ReLU(),
                            nn.Linear(in_features=10, out_features=3, bias=True),
                            nn.LogSoftmax()
                           )

In [6]:
learn = create_cnn(data, models.resnet50, metrics=accuracy, custom_head=custom_head)

In [9]:
learn.unfreeze()

In [10]:
learn.fit_one_cycle(1, max_lr=slice(1e-6,1e-3))

epoch,train_loss,valid_loss,accuracy
1,0.216016,0.467687,0.817645


/home/as3ek/.local/lib/python3.6/site-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [11]:
learn.save('resnet_10_output_1_epoch')

In [32]:
class ResNet50Bottom(nn.Module):
    def __init__(self, original_model):
        super(ResNet50Bottom, self).__init__()
        self.features = nn.Sequential(*(list(learn.model.children())[:-1] + list(list(learn.model.children())[1])[:-3]))
        
    def forward(self, x):
        x = self.features(x)
        return x

model_10_features = ResNet50Bottom(learn.model)

In [82]:
train_df = data.train_ds.to_df()

In [ ]:
bs = 16
data_model_trn = pd.DataFrame(columns = list(range(10)).append('fname'))

dl_iter = iter(learn.data.train_dl)
dl = next(dl_iter)
for index, name in enumerate(train_df['x']):
    if ((index % bs == 0) and (index != 0)):
        dl = next(dl_iter)
    image_batch, label_batch = dl
    i = index % bs
    image = image_batch[i].reshape(1, 3, 512, 512)
    label = label_batch[i].reshape(-1)
    
    output = model_10_features(image)
    
    if (index % 100 == 0):
        print (index)
    
    a = output[0].cpu().detach().numpy()
    tmp = pd.DataFrame(data=a).T
    tmp['fname'] = [name]
    data_model_trn = pd.concat([data_model_trn, tmp], ignore_index=True)

0
100
200
300
400
500


In [73]:
# we start by extracting a batch then choosing one image and label from that batch
# i is the index of the chosen image
i = 13

image_batch, label_batch = next(iter(learn.data.train_dl))
image = image_batch[i].reshape(1, 3, 512, 512)
label = label_batch[i].reshape(-1)

# running the forward pass
out = model_10_features(image)

In [81]:
for i, j in enumerate(train_df['x']):
    print(j)
    break

train/Celiac/C13-65_04___2000_4432___0_0.jpg
